In [13]:
import argparse
import py_trees
import sys
import time
"""
import threading , math , time , datetime , os , pickle , builtins , types
from math import radians
from time import sleep
from datetime import datetime
from pytz import timezone
mst = timezone('MST')
# ~~ Special ~~
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
import py_trees
"""
import py_trees.blackboard as PTBlackboard
from py_trees.common import Status

#from py_trees.tests import Timeout_Success

#from utils import ( mindex , vec_unit , condition_false , combine_poses , is_matx_list , HeartRate , TimerQueue )
#selector
"""
content = "Higher priority switching and interruption in the children of a selector.\n"
content += "In this example the higher priority child is setup to fail initially,\n"
content += "falling back to the continually running second child. On the third\n"
content += "tick, the first child succeeds and cancels the hitherto running child.\n"
"""
#sequence:
"""
"A sequence is populated with 2-tick jobs that are allowed to run through to completion.\n"
"""
class HeartRate: 
    """ Sleeps for a time such that the period between calls to sleep results in a frequency not greater than the specified 'Hz' """
    # NOTE: This fulfills a purpose similar to the rospy rate
    
    def __init__( self , Hz ):
        """ Create a rate object with a Do-Not-Exceed frequency in 'Hz' """
        self.period = 1.0 / Hz; # Set the period as the inverse of the frequency , hearbeat will not exceed 'Hz' , but can be lower
        self.last = time.time()
    
    def sleep( self ):
        """ Sleep for a time so that the frequency is not exceeded """
        elapsed = time.time() - self.last
        if elapsed < self.period:
            time.sleep( self.period - elapsed )
        self.last = time.time()

        
def run_BT_until_done( rootNode , 
                       N = 1500 , tickPause = 0.25 , 
                       breakOnFailure = 1 , breakOnSuccess = 1 , 
                       Nverb = 100 ):
    """ Tick root until `maxIter` is reached while printing to terminal """
    
    print( "### RUN BT" , rootNode.name , "###\n" )
        
    bgn     = time.time()
    elapsed = 0.0

    # 0. Setup
    rootNode.setup_with_descendants()
    pacer = HeartRate( Hz = 1/tickPause ) # metronome
    # 1. Run
    for i in range( 1 , N+1 ):
        try:
            rootNode.tick_once()

            if Nverb > 0 and i % Nverb == 0:
                print("\n--------- Tick {0} ---------\n".format(i))
                print("\n")
                print( py_trees.display.unicode_tree( root        = rootNode ,
                                                      show_status = True     ) )

            if  breakOnFailure  and  ( rootNode.status == Status.FAILURE ):
                print( "Root node" , rootNode.name , "failed!\n" )
                break
            elif  breakOnSuccess  and  ( rootNode.status == Status.SUCCESS ):
                print( "Root node" , rootNode.name , "succeeded!\n" )
                break
            else:
                if Nverb > 0 and i % Nverb == 0:
                    print( "Root node" , rootNode.name , "status:" , rootNode.status )
                pacer.sleep()

        except KeyboardInterrupt:
            break
    if Nverb > 0:
        print( py_trees.display.unicode_tree( root        = rootNode ,
                                              show_status = True     ) )
    elapsed = time.time() - bgn
    print("\nRun completed! with status" , rootNode.status , "after" , elapsed , "seconds.\n\n")
    rootNode.terminate( rootNode.status ) # HACK required because the tree doesn't complete sometimes
            
    print( "\n### BT SESSION OVER ###" )


def create_root():
    root = py_trees.composites.Sequence("Sequence")
    for action in ["Action 1", "Action 2", "Action 3"]:
        success_after_two = py_trees.behaviours.Count(name=action,
                                                      fail_until=0,
                                                      running_until=1,
                                                      success_until=10)
        root.add_child(success_after_two)
        
        

    root2 = py_trees.composites.Selector("Selector")
    for action in ["Action A", "Action B"]:
        success_after_two2 = py_trees.behaviours.Count(name=action,
                                                  fail_until=2,
                                                  running_until=2,
                                                  success_until=10)
        
        root2.add_child(success_after_two2)
    success_after_two3 = py_trees.behaviours.Count(name="Action C",
                                                  fail_until=0,
                                                  running_until=1,
                                                  success_until=10)
    root2.add_child(success_after_two3)
    #always_running = py_trees.behaviours.Running(name="Running")
    #root.add_children([success_after_two2, always_running])
    #root2.add_children([success_after_two2])
    root.add_children([root2])
    return root


##############################################################################
# Main
##############################################################################

def main():
    """
    Entry point for the demo script.
    """
    #args = command_line_argument_parser().parse_args()
    #print(description())
    #py_trees.logging.level = py_trees.logging.Level.DEBUG

    root = create_root()

    #py_trees.display.render_dot_tree(root)  #output tree images

    ####################
    # Execute
    ####################
    root.setup_with_descendants()
    
   
    i=1
    #for i in range(1, 6):
    """"   
    run_BT_until_done( rootNode=root , 
                       N = 1500 , tickPause = 0.25 , 
                       breakOnFailure = 1 , breakOnSuccess = 1 , 
                       Nverb = 100 )
    
    #rootNode.status == Status.SUCCESS
    """
    while root.status==Status.INVALID or root.status==Status.RUNNING: #or root.status==py_trees.common.Status.INVALID :
            try:
                #print("\n--------- Tick {0} ---------\n".format(i))
                #print(root.Status)
                root.tick_once()

                #print("\n")
                #print(py_trees.display.ascii_tree(root=root, show_status=True))
                i=i+1

                #time.sleep(1.0)
                #status=root.update()
                #print(root.status)
            
            except KeyboardInterrupt:
                print("interrupt")
                break
    print(py_trees.display.ascii_tree(root=root, show_status=True))            
    print("complete")

In [14]:
main()

[-] Sequence [o]
    --> Action 1 [o]
    --> Action 2 [o]
    --> Action 3 [o]
    [o] Selector [o]
        --> Action A [x]
        --> Action B [x]
        --> Action C [o]

complete
